## TOPICS COVERED

* How to import a module from anotehr jupyter notebook file (.ipynb)
* Use the .read() method from the spark session to create a DataFrame from a local file (.csv, .json etc)
* Set a custom schema to the DataFrame through the sparkSession.read.json() operation
>* Use StructField, StructType and various base types IntegerType, StringType from pyspark.sql.types to change the schema of a json being imported into a spark DataFrame.
>* Use StructField(fieldName, StringType(), True) to define json type of a field of String type. For int type use IntegerType()
>* Use a list of StructField entries representing all the columns of the json file
>* Create the StructType schema (i.e. StructType(fields=list_of_StructFields)) and pass it as "schema" parameter to the sparksession read method (i.e. .read.json(jsonFile,schema=new_json_schema))
>* The DataFrame returned above will have specific data types instead of Strings. Validate it using sparkDF.printSchema()
* Selecting / filtering data from DataFrame.
>* sparkSessn.sql(...) : Use direct Select SQL statements on the dataframe using sparkDF.sql(SQL_SELECT_STMT) e.g. "select col1, col2 from table1 where age > 60 and duration < 120"
>* sDF.select(...).where(...) : This returns subset of DataFrame(sDF) with specified columns(in select clause)  satisfying the condition passed is where clause.
>* sDF.filter(...) : This returns subset of DataFrame(sDF) with all columns satisfying the condition passed as filter e.g. "age > 18"
* Use "groupBy('col1')" to group the data by the specified column in a DataFrame and run aggregate functions on the groupby object.

#### pyspark API Documentation:
* http://spark.apache.org/docs/latest/
* http://spark.apache.org/docs/latest/ml-guide.html
* https://spark.apache.org/docs/latest/api/python/

#### Enabling Jupyter shell to print multiple results form a single shell

In [ ]:
## Enable the shell to print multiple results (instead of only the last result)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###### Standard Imports for Spark
* from pyspark.sql import SparkSession   ### Starting point for Spark - used to get the <b>spark session instance</b> using builder pattern
* from pyspark.sql.types import StructField, IntegerType, StringType, StructType   ### For using <b>custom json schema</b>
* from pyspark.sql.functions import concat, countDistinct, mean, avg, stddev       ### For using <b>aggregate groupby functions</b> with Spark DataFrames
* from pyspark.sql.functions import col ### Returns a <b>column with name</b> specified by the parament
* from pyspark.sql.functions import date_format, year,month,dayofmonth,hour,dayofweek,dayofyear,weekofyear,format_number   ### For handling <b>Date and Timestamp</b>
* from .defs.Chinmay_Utilities import getCallLogXmlFromSuperbackup, getSparkDFfromPandasDF, getJsonFromSparkDF, printTextFile, getPandasDFfromSparkDF  ### My personal <b>utilities from another ipynb</b> jupyter notebook

###### Import modules form another ipynb (jupuyter notebook written by me

In [ ]:
import ipynb.fs  # Boilerplate required

# Do a full import
# from .full.Chinmay_Utilities import foo

# Do a definitions-only import

### Spark supproting methods
from .defs.Chinmay_Utilities import getCallLogXmlFromSuperbackup, getSparkDFfromPandasDF, \
                                        getJsonFromSparkDF, getPandasDFfromSparkDF, getMaskedSparkDF

### Printing methods
from .defs.Chinmay_Utilities import printTextFile, getBold, getUnderlined, getColorInverted, printHighlighted


# We can "import ipynb.fs.defs.Chinmay_Utilities" instead of two imports "import ipynb.fs" followed by ".defs.Chinmay_Utilities"

###### Import the tool from Chinmay as an alternate to import of ipynb file
* Convert the ipynb files into .py python scripts, add them to python path and import that file in the desired code

In [ ]:
import sys
sys.path.append('C:/Users/nishita/exercises_udemy')
from tools.chinmay_tools import printHighlighted

In [ ]:
printHelpOnFormattedText()

###### All Spark Imports

In [ ]:
from pyspark.sql import SparkSession   ### Starting point for Spark - used to get the <b>spark session instance</b> using builder pattern
from pyspark.sql.types import StructField, IntegerType, StringType, StructType   ### For using <b>custom json schema</b>
from pyspark.sql.functions import concat, countDistinct, mean, avg, stddev, col    ### For using <b>aggregate groupby functions</b> with Spark DataFrames
from pyspark.sql.functions import date_format, year,month,dayofmonth,hour,dayofweek,dayofyear,weekofyear,format_number   ### For handling <b>Date and Timestamp</b>

###### EXECUTE UPTO THIS POINT ALWAYS
###### ----------------- The below points can be executed based on requirement ------------------

## Spark DataFrame Basics Experiments
###### (Section 8.1 to 8.4))

###### Refer Documentation for pyspqrk.sql package at https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

In [ ]:
from pyspark.sql import SparkSession

#### Getting help on a method in a builder pattern

In [ ]:
# To get the help of a method / attribute in a builder pattern, 
#     split the pattern just before that method
#     set a variable with the builder pattern result just before that method call, 
#         so that the method call can be performed on the variable.
#     Now execute (SHIFT+ENTER) the help syntax i.e. "method?"" NOT "method()?"" on that variable
#     
# Below is an example for getting help on getOrCreate() method in "SparkSession.builder.appName('Basics').getOrCreate()"
#     
bld = SparkSession.builder.appName('Basics')
# bld.getOrCreate??  ### Uncomment this line to get the help ("?") and code implementation ("??")
#     
# Here we can not use "SparkSession.builder.appName('Basics').getOrCreate?", because there is a use input involved (i.e. parameter of appName())

In [ ]:
# Uncomment specific function below and run this shell to get help
# getCallLogXmlFromSuperbackup?
# getSparkDFfromPandasDF?
# getJsonFromSparkDF?
# printTextFile??
# getPandasDFfromSparkDF?


###### Read a json using SparkSession and analyse the result databrame

In [ ]:
#Get a SparkSession
sparkSesnBasic = SparkSession.builder.appName('Basics').getOrCreate()

my_json_people = "test_data/people.json"
printTextFile(my_json_people)
sdf = sparkSesnBasic.read.json(my_json_people)

# Types of files that can be read csv/format/jdbc/json/load/option/options/orc/parquet/schema/table/text

In [ ]:
#printSchema automatically decides the schema based on data.
sdf.printSchema()

sdf.columns

sdf.describe

sdf.describe()

# describe() given summary of numeric columns in the dataframe
sdf.describe().show()

###### Modify the JsonSchema using a user defined schema

In [ ]:
# Many times spark can not determine the data types in a json correctly and specifies each of the fields as String.
# In this case we can define a schema and attach it to the json
from pyspark.sql.types import StructField, IntegerType, StringType, StructType

In [ ]:
# StructField(field_name, field_type(), is_field_nullable)
# To enforce the user defined scheme to a json pass a list of structfields one for each column
data_schema = [StructField ('age', IntegerType(), False), 
                StructField('name', StringType(), True)]

final_type = StructType(fields=data_schema)

final_type
final_type["age"]
final_type["name"]

In [ ]:
StructField?

###### Read the same json using the user defined schema (earlier it was the default one)

In [ ]:
printTextFile(my_json_people)
sdf2 = sparkSesnBasic.read.json(my_json_people, schema=final_type)

sdf2.printSchema()
sdf2.show()

###### Refer back the spark dataframe with builtin default schema

In [ ]:
sdf.printSchema()
sdf.show()

### Invoking a method to get the sample call log using a module from another ipynb file in the same folder
* ###### sparkCallSession.createDataFrame(p_df) --> converts pandas dataframe into spark dataframe
* ###### s_df.select("*").toPandas() --> converts spark dataframe into pandas dataframe

In [ ]:
# Get the call log xml data in a Pandas DataFrame
dfCallLogs = getCallLogXmlFromSuperbackup("test_data/calllogs_20200512130135.xml")   ## Calling from Chinmay_Utilities.ipynb

# Convert the Pandas DataFrame into Spark DataFrame
sdfCallLogs = getSparkDFfromPandasDF(dfCallLogs)   ## Calling from Chinmay_Utilities.ipynb

#### Using Select statements with Spark
* We can use limited select statement (upto selection of columns, adding computed columns and without any where clause)
* * spark dataframe can use complex where clause as explained below
* To use full version of select sql along with where clause we need to register the Spark DataFrame as a table using the method below.
* * sparkDF.createOrReplaceTempView(pseudo_tableView_name)
* * This is used only with sparkSessn.sql()

In [ ]:
# Register the spark dataframe as a table/view to be used like standard sql using sparkSession.sql()
sdfCallLogs.createOrReplaceTempView("call_logs")

##### Using pure SQL with DataFrames
* Register the dataframe as a table (sdf.createOrReplaceTempView(table1)) and
* Use sparkSession1.sql(SQL_STMT_using_table1)

##### Filtering using SQL
* ###### sparkSesn.sql(full_sql)
* ###### This works but supports limited where clause

##### Using pure SQL with DataFrames
* Register the dataframe as a table and
* Use sparkSession1.sql(SQL_STMT_table) E.g. sparkSesnBasic.sql("SELECT * FROM call_logs WHERE dur > 100 ORDER BY dur DESC").show()
* The where clause of this sql is not supporting LIKE clause
* This complex where clause (LIKE clause) is possible through direct "where" clause on spark dataframe (next statement)

###### Quick filtering: (one of the two alternate ways on sparkDF)
* ###### sparkDF.select(*).where(my_condition)
* ###### sparkDF.filter(my_condition)

In [ ]:
my_where_clause = "upper(name) like '%SEEC%QA%' AND dur > 50"

In [ ]:
# Querying the Spark DataFrame directly with WHERE clause
# This sort of complex where clauses (e.g. LIKE clauses) are not possible with pandas dataframe

# sdfCallLogs.select("*").where("upper(name) like '%SEEC%QA%'").show()

# Convert the filtered data into pandas data frame which can be processed or outputted into a file
df_qa = sdfCallLogs.select("*").where(my_where_clause).toPandas()
df_qa

In [ ]:
sdfCallLogs.filter(my_where_clause).toPandas()

In [ ]:
# Rename the columns before converting to Pandas DataFrame
df_qa2 = sdfCallLogs.filter(my_where_clause).select('name', 'number', 'dur', 'time') \
    .withColumnRenamed('dur', 'Duration (Sec)').withColumnRenamed('name', 'Name')\
    .withColumnRenamed('time', 'Date').withColumnRenamed('number', 'Phone Number').toPandas()

# Below line inserts a new column with value double of the current 'dur' columns value
df_qa2B = sdfCallLogs.filter(my_where_clause).select('name', 'number', 'dur', 'time') \
            .withColumn('double_duration',sdfCallLogs['dur']*2).toPandas()

# To show a pd.DataFrame without column index
df_qa2.style.hide_index()
df_qa2B.style.hide_index()

# To write an dataframe to an excel file without index column
df_qa2.to_excel('1.xlsx',index=False)

In [ ]:
sdfCallLogs.filter(my_where_clause).filter((sdfCallLogs['dur']>300) & ~(sdfCallLogs['dur']<1000)).show()

In [ ]:
sdfCallLogs.filter(my_where_clause).filter('dur>300 and dur>=1000').show()

###### Convert the Spark DataFrame into Json
###### Convert a Pandas DataFrame into Json by first converting into a Spark DataFrame

In [ ]:
getJsonFromSparkDF(sdfCallLogs)   ## Calling from Chinmay_Utilities.ipynb

In [ ]:
# To get Json from a pandas DataFrame fist convert itinto a Spark DataFrame and then get Json from it
getJsonFromSparkDF(getSparkDFfromPandasDF(dfCallLogs))   ## Calling from Chinmay_Utilities.ipynb

##### Check types of DataFrame, Columns and displaying selected columns as DataFrame

In [ ]:
type(sdf)
type(sdf['Age'])
#sdf['Age'].show()         # This line does nto work as we can not display columns
sdf.select('Age').show()   # This returns a dataframe of selected columns
type(sdf.select('Age'))

sdf[sdf['Age']>0].show()    # This works similar to regular pandas dataframe filtering

##### Display rows form top of dataframe

In [ ]:
sdf.head(10) # display atmost 10 rows from top of df

##### Renaming a column and Inserting a computed column

In [ ]:
sdf1 = sdf.withColumnRenamed('age', 'old_age')
type(sdf1)
sdf1.show()

##### Renaming a column and Inserting a computed column

In [ ]:
sdf2 = sdf.withColumn('double_age',sdf['age']*2)
type(sdf2)
sdf2.show()

## Experimenting with GroupBy and Aggregates
###### (Section 8.5))

In [ ]:
from pyspark.sql import SparkSession
sparkSesnGrpby = SparkSession.builder.appName("chin_groupby").getOrCreate()

my_csv_sales_info = "test_data/sales_info.csv"
sdf_sales = sparkSesnGrpby.read.csv(my_csv_sales_info, inferSchema=True, header=True)
# allow spark to assume first row as the column names and to decide the data type from data value

sdf_sales.printSchema()

sdf_sales.show()

In [ ]:
sdf_sales.groupBy("Company").sum().collect()  # Returns a list of Row objects which can be used in a function

In [ ]:
group_data_by_company = sdf_sales.groupBy('Company')
group_data_by_company.sum().show()
sdf_sales.groupBy('Company').sum().show()

In [ ]:
sdf_sales.columns
getJsonFromSparkDF(group_data_by_company.sum())  # using function from Chinmay_Utilities.ipyn

##### Using ".agg" function (two params - 1: col name, 2:aggregate func name)
* ###### Getting aggregate across entire table (or data frame)
* ###### Getting aggregates for each unique value of the group by column
* * The ".agg(col_name : agg_func_name)" can also be used as ".agg_func(col_name)"
* * E.g:  sdf.groupby("Company").agg({"Company" : "max"}) gives same result as sdf.groupby("Company").max("Company")
* * VArious pre defined aggregate functions are: mean / min / max / sum / count etc..

In [ ]:
sdf_sales.agg({"Sales":"count"}).show()
sdf_sales.groupBy("Company").agg({"Sales":"count"}).show()

# You can try with other aggregate functions mean / min / max / sum / count etc..

# When used directly on a DataFrame, it gives the overall result across the dataframe
# When used on top of a groupby result of a DataFrame, it gives the aggregate results for each unique value of groupby columna

In [ ]:
group_data_by_company = sdf_sales.groupBy('Company') ### Both are same

group_data_by_company.agg({"Sales":"max"}).show()  # parameter format("col_name" : "aggregate_func_name")
# Various aggregate functions are: mean / min / max / aum / count etc..
## This agg() format is more generalized and we can put in a for loop by passing aggregate func name in a param

group_data_by_company.sum("Sales").show()
## This is more Rigid as we need to call the aggregate functions explicitly

### sdf_sales.groupBy('Company') can also be used in place of group_data_by_company

In [ ]:
sdf_sales.groupBy("Company")
# Aggregators on top of GroupedData returns a spark DataFrame for our consumption
# Various gorubby aggregator functions: mean/sum/max/min/count
sdf_sales.groupBy("Company").mean().withColumnRenamed('avg(Sales)','Average Sales').show()

In [ ]:
group_data_by_company.agg({"Sales":"mean"}).show()
group_data_by_company.agg({"Sales":"max"}).show()
group_data_by_company.agg({"Sales":"min"}).show()
group_data_by_company.agg({"Sales":"sum"}).show()
group_data_by_company.agg({"Sales":"count"}).show()


In [ ]:
sdf_sales.groupBy('Company').mean("Sales").show()

### Using Functions from Spark
* Compute aggregate operations within ".select" clause using functions
* Cormat the result through a second select on first result
* Sort data in a dataframe using column name (by default ascending order)
* Sort data in dataframe in descending (using a descend ordered column)
* Sort data in dataframe with different orders for different columns (uses column but not names, desc() applied to individual columns as needed)

We can import a function from pyspark.sql.functions and use that function inside a SELECT statement like sdf.select(avg('Sales')),show() -- very useful in handling data, timestamps etc
* sdf.select(avg('Sales'),alias("Average Sales")),show()  -- This renames the column heading of the result
* Other functions such as ".corr" for correlatin, ".stddev" for standard deviation etc can eb used as well.

In [ ]:
from pyspark.sql.functions import concat, countDistinct,avg, stddev
from pyspark.sql import SparkSession

In [ ]:
print(getColorInverted(getUnderlined(getBold("Created a spark session and Loaded Spark DataFrame from a csv file"))))

sparkSesn3 = SparkSession.builder.appName("chin_groupby").getOrCreate()

# REPEATING DEFINITION (for readability)
my_csv_sales_info = "test_data/sales_info.csv"

# allow spark to assume first row as the column names and to decide the data type from data value
sdf_sales = sparkSesn3.read.csv(my_csv_sales_info, inferSchema=True, header=True)

sdf_sales.printSchema()

sdf_sales.show()

##### Apply aggregation functions and format the result
* Aggregation functions are applied through functios within select clause of dataframe
* Format the orderBy ressult
* * sdfaggregted = sdf.select(aggr_func('col_name'))
* * sdfFinal = sdf_aggregted.select(format_number('col_name', n))     # n = decoimal precision

In [ ]:
printHighlight("Unique Sales Records")

In [ ]:
# Number of distinct valeus in 'Sales' column
printHighlighted("Unique Sales Records")
sdf_sales.select(countDistinct('Sales')).show()

printHighlighted("Unique Sales Records - Column renamed")
sdf_sales.select(avg('Sales').alias('Unique Sales Count')).show()

printHighlighted("Total Sales Records")
sdf_sales.agg({'Sales':'count'}).show()

printHighlighted("Average Sales Records")
sdf_sales.select(avg('Sales')).show()

printHighlighted("Standard deviation Sales Records")
sdf_sales.select(stddev('Sales')).show()

printHighlighted("Standard deviation Sales (formatted precision)")
printHighlighted("\t * format_number() should be applied through a second select on the result data frame from the first select")

from pyspark.sql.functions import format_number
sdf_sales.select(stddev('Sales').alias('stddev')).select(format_number('stddev',2).alias('std_dev')).show()

##### Sorting data in a DataFrame
* * ASCENDING:  sdf.orderBy(col_name)
* * DESCENDING: sdf.orderBy(df[col_name].desc())
* CASE#1: Ascending (default) order of a single column [use orderBy on a column name]
* CASE#2: Ascending (default) order of multiple columns [use orderBy on a column name]
* CASE#3: Descending order [Use orderBy on a column instead of a column_name], use desc() on the column
* CASE#4: Different order for different columns order [Use orderBy on a multiple columns instead of a column_names], use desc() on the columns


In [ ]:
printHighlighted("Original dataframe")
sdf_sales.show()
printHighlighted("CASE#1: Sorting a dataframe based on one column")
sdf_sales.orderBy('Sales').show()
printHighlighted("CASE#2: Sorting a dataframe based on two columns (Look for 'Sales' of 350)")
sdf_sales.orderBy('Sales', 'Person').show()

In [ ]:
printHighlighted("CASE#3: Sorting (Descending order) a dataframe based on one column")
sdf_sales.orderBy(sdf_sales['Sales'].desc()).show()

printHighlighted("CASE#4: Sorting (different order for different columns) a dataframe based on one column (Look for 'Sales' of 350)")
sdf_sales.orderBy(sdf_sales['Sales'].desc(), sdf_sales['Person']).show()

### Handling Missign data in spark DataFrames
3 ways to handle
* Delete the entrire records where value is mising for ateleast one column [sdf.na.drop]
* Set the missign vales to "null"
* Set the missign vales to a pre-decided value
* Set the missign vales to a computed value

In [ ]:
printHighlighted("Created a spark session and Loaded Spark DataFrame from acsv file")
sparkSesn4 = SparkSession.builder.appName("chin_miss").getOrCreate()

my_csv_contains_null = "../Python-and-Spark-for-Big-Data-master/Spark_DataFrames/ContainsNull.csv"
# allow spark to assume first row as the column names and to decide the data type from data value

sdf_missing = sparkSesn3.read.csv(my_csv_contains_null, inferSchema=True, header=True)
sdf_missing.printSchema()
sdf_missing.show()

###### Dropping null records
* sdf.na.drop(how='any') --> drops rows with null in any field
* sdf.na.drop() --> same result as sdf.na.drop(how='any')
* sdf.na.drop(how='all') --> drops rows with null in all the fields - a very rare case
* sdf.na.drop(threes=n) --> Keeps rows with atleast n non-null fields, i.e. drops rows with upto n-1 null fields
* * if n > number of cols, then all rows are dropped - results is an empty data frame
* * if n < 2, then no row is dropped - results is same as sdf.show()
* sdf.na.drop(subset='MyCol') --> drops rows with null in column 'MyCol'

In [ ]:
str_temp = "* sdf.na.drop(how='any') --> drops rows with null in any field"
printHighlighted(str_temp)
sdf_missing.na.drop(how='any').show() #drop if any field is null

str_temp = "* sdf.na.drop() --> same result as sdf.na.drop(how='any')"
printHighlighted(str_temp)
sdf_missing.na.drop().show()   #.drop() is same as .drop(how='any') i.e. any non-null field

str_temp = "* sdf.na.drop(how='all') --> drops rows with null in all the fields - a very rare case"
printHighlighted(str_temp)
sdf_missing.na.drop(how='all').show() #drop if all fields are null

str_temp = "* sdf.na.drop(threes=2) --> Keeps rows with atleast 2 non-null fields, i.e. drops rows with upto 2-1 null fields"
printHighlighted(str_temp)
sdf_missing.na.drop(thresh=2).show() # drop if there are less than n (here less than 2) null fields in a row

In [ ]:
str_temp = "* sdf.na.drop(subset='Name') --> drops rows with null in column 'Name'"
str_temp += "\n* sdf.na.drop(subset=('Name', 'Sales')) --> drops rows with null in any of the two columns 'Name' or 'Sales'"
str_temp += "\n* sdf.na.drop(subset='Name') --> EQUIVALENT to sdf.na.drop(subset=('Name'))"
printHighlighted(str_temp)
sdf_missing.na.drop(subset=('Name')).show()
sdf_missing.na.drop(subset=('Name', 'Sales')).show()

###### Replacing null valeus with a fixed value
* sdf.na.fill('MY STRING') --> Replaces all null values in all the String columns with given string
* sdf.na.fill(n) --> Replaces all null values in Numeric columns with given number
* sdf.na.fill('MY STRING', subset=('col1', 'col2')) --> replaces the null values with given string only for the list of columns supplied
* * A passed in column is ignored if its type does not match with the type of value passed as first parameter.
* * .fill('val', subset=('col1', 'col2')) is EQUIVALENT to .fill('val', ['col1', 'col2'])
* * .fill('val', subset='col1') is EQUIVALENT to .fill('val', subset=('col1')) - is a special case of above one

In [ ]:
#sdf_missing.na.fill(999).na.fill('CHINMAY').show()
sdf_missing.na.fill('ANAND', subset='Name').show()
sdf_missing.na.fill('ANAND', ['Name', 'Sales']).show()  # integer column is ignored as passed value is of string type

###### Replace the null values in a numeric column with it's mean i.e mean of non-null values in that column

In [1]:
from pyspark.sql.functions import mean, avg

# mean_sales = sdf_missing.agg({'Sales':'mean'}).collect()[0][0]
# mean_sales = sdf_missing.select(avg(sdf_missing['Sales'])).collect()[0][0]
mean_sales = sdf_missing.select(mean(sdf_missing['Sales'])).collect()[0][0]
sdf_missing.na.fill(mean_sales, ['Sales']).show()

NameError: name 'sdf_missing' is not defined

### Handling Dates and Timestamps in Spark DataFrame (for apple stock)
* ###### Ref: https://obstkel.com/spark-sql-date-functions
* ###### Ref: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=date

In [ ]:
# from pyspark.sql import SparkSession
sparkSessnStock = SparkSession.builder.appName("chin_date").getOrCreate()
my_csv_app_stocks = "test_data/appl_stock.csv"
sdf_stock = sparkSessnStock.read.csv(my_csv_app_stocks, header=True, inferSchema=True)
sdf_stock.columns
sdf_stock['Date', 'Open'].head(3)
#sdf_stock.show()

###### Find Average closing price of the stock per year
1. Create a new column 'Year' in the DF with the value of year (use sdf.withColumn())
2. Call ".groupBy('year).avg()" on result of step-1 -- each averaged numeric column 'col_name' will be named avg(col_name)
3. Call ".select('Year', 'avg(Close)')" on result of step-2
4. Call a second select with same set of columns but the formatting_needed_columns passed through format_number()
    * ".select('Year', format_number(avg(Close), nPrecision).alias('Avg Close'))"

In [ ]:
printHighlighted("Added computed columns 'Year', 'Month', 'Day', 'Week Day' to the table")

sdf_stock_new = sdf_stock.select(
    year('Date').alias('Year'),
    month('Date').alias('Month'),
    dayofmonth('Date').alias('Day'),
    dayofweek('Date',).alias('Week Day'),
    date_format('Date','yyyy-MM-dd'),'*'
)

###### Different aggregations on different columns on a groupby object and formatting the result and displaing in multi-sorted order

In [ ]:
printHighlighted("Added computed columns 'Year', 'Month', 'Day', 'Week Day' to the table")

grpby_year_stock = sdf_stock_new.groupBy('Year')
agg_stock_dataframe = \
    grpby_year_stock.agg({'Close':'avg', 'Open':'min', 'High':'max', 'Low':'min'})\
                    .select('Year', \
                        format_number('avg(Close)', 2).alias('avg_close'), \
                        format_number('min(Open)', 2).alias('min_open'), \
                        format_number('max(High)', 2).alias('max_high'), \
                        format_number('min(Low)', 2).alias('min_low'))
printHighlighted("Grouped by 'Year'")
agg_stock_dataframe.orderBy('Year').show()

printHighlighted("Grouped by 'Year', sorted by 'Year' ascending and 'avg_close' descending - cascaded orderBy")
agg_stock_dataframe.orderBy('Year').orderBy(agg_stock_dataframe['avg_close'].desc()).show()  ##cascaded orderby

printHighlighted("Grouped by 'Year', sorted by 'Year' ascending and 'avg_close' descending - 1st as col name, 2nd as column")
agg_stock_dataframe.orderBy('Year',agg_stock_dataframe['avg_close'].desc()).show()  ## mixed ordering

printHighlighted("Grouped by 'Year', sorted by 'Year' ascending and 'avg_close' descending - both as columns")
agg_stock_dataframe.orderBy(agg_stock_dataframe['Year'],agg_stock_dataframe['avg_close'].desc()).show() # descending order